In [2]:
#unsynchronized threads - no lock/acquire/release - or join to wait for 
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    def run(self):
        print "Starting " + self.name
        print_time(self.name, self.counter, 5)
        print "Exiting " + self.name

def print_time(threadName, delay, counter):
    while counter:
        if exitFlag:
            threadName.exit()
        time.sleep(delay)
        print "%s: %s" % (threadName, time.ctime(time.time()))
        counter -= 1

# Create new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# Start new Threads
thread1.start()
thread2.start()

print "Exiting Main Thread"

Starting Thread-1
Starting Thread-2
Exiting Main Thread
Thread-1: Sat May  6 09:05:38 2017
Thread-2: Sat May  6 09:05:39 2017Thread-1: Sat May  6 09:05:39 2017

Thread-1: Sat May  6 09:05:40 2017
Thread-2: Sat May  6 09:05:41 2017
Thread-1: Sat May  6 09:05:41 2017
Thread-1: Sat May  6 09:05:42 2017
Exiting Thread-1
Thread-2: Sat May  6 09:05:43 2017
Thread-2: Sat May  6 09:05:45 2017
Thread-2: Sat May  6 09:05:47 2017
Exiting Thread-2


In [1]:
#Synchronized threads - lock/acquire/release for locking mechanism - join() to wait for all threads to finish

import threading
import time

class myThread (threading.Thread):
    def __init__(self, threadID, name, sleepdelay):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.sleepdelay = sleepdelay
    def run(self):
        print "Starting " + self.name
        # Get lock to synchronize threads
        threadLock.acquire()
        print_time(self.name, self.sleepdelay, 3)
        # Free lock to release next thread
        threadLock.release()

def print_time(threadName, delay, counter):
    while counter:
        time.sleep(delay)
        print "%s: %s" % (threadName, time.ctime(time.time()))
        counter -= 1

threadLock = threading.Lock()
threads = []

# Create new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# Start new Threads
thread1.start()
thread2.start()

# Add threads to thread list
threads.append(thread1)
threads.append(thread2)

# Wait for all threads to complete
for t in threads:
    t.join()
print "Exiting Main Thread"

Starting Thread-1
Starting Thread-2
Thread-1: Sat May  6 09:05:29 2017
Thread-1: Sat May  6 09:05:30 2017
Thread-1: Sat May  6 09:05:31 2017
Thread-2: Sat May  6 09:05:33 2017
Thread-2: Sat May  6 09:05:35 2017
Thread-2: Sat May  6 09:05:37 2017
Exiting Main Thread


In [1]:
import Queue
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    def run(self):
        print "Starting " + self.name
        process_data(self.name, self.q)
        print "Exiting " + self.name

def process_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            data = q.get()
            queueLock.release()
            print "%s processing %s" % (threadName, data)
        else:
            queueLock.release()
        time.sleep(1)

threadList = ["Thread-1", "Thread-2", "Thread-3"]
nameList = ["One", "Two", "Three", "Four", "Five"]
queueLock = threading.Lock()
workQueue = Queue.Queue(10)
threads = []
threadID = 1

# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue
queueLock.acquire()
for word in nameList:
    workQueue.put(word)
queueLock.release()

# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print "Exiting Main Thread"


Starting Thread-1
Starting Thread-2
Starting Thread-3
Thread-3 processing One
Thread-1 processing Two
Thread-2 processing Three
Thread-3 processing Four
Thread-1 processing Five
Exiting Thread-2
Exiting Thread-3
 Exiting Thread-1
Exiting Main Thread


Semaphore vs Mutex:

Semaphore: Use a semaphore when you (thread) want to sleep till some other thread tells you to wake up. Semaphore 'down' happens in one thread (producer) and semaphore 'up' (for same semaphore) happens in another thread (consumer) e.g.: In producer-consumer problem, producer wants to sleep till at least one buffer slot is empty - only the consumer thread can tell when a buffer slot is empty.

Mutex: Use a mutex when you (thread) want to execute code that should not be executed by any other thread at the same time. Mutex 'down' happens in one thread and mutex 'up' must happen in the same thread later on. e.g.: If you are deleting a node from a global linked list, you do not want another thread to muck around with pointers while you are deleting the node. When you acquire a mutex and are busy deleting a node, if another thread tries to acquire the same mutex, it will be put to sleep till you release the mutex.

Process vs threads:
Both processes and threads are independent sequences of execution. The typical difference is that threads (of the same process) run in a shared memory space, while processes run in separate memory spaces.

I'm not sure what "hardware" vs "software" threads might be referring to. Threads are an operating environment feature, rather than a CPU feature (though the CPU typically has operations that make threads efficient).

Erlang uses the term "process" because it does not expose a shared-memory multiprogramming model. Calling them "threads" would imply that they have shared memory.

In [3]:
#Share data between processes in python using queue or pipe:

from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print q.get()    # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


In [2]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    p = Pool(5)
    print(p.map(f, [1, 2, 3]))

[1, 4, 9]


In [5]:
a,b=[2,3]
print a
print b

2
3


In [11]:
a=[]
'asd' if a else 'qwe'

'qwe'

In [19]:
import re
re.findall(r'(\d+\.\d+\.\d+\.\d+)','10.5.5.234 12.34 45.23 23.45.23.45')


['10.5.5.234', '23.45.23.45']

In [13]:
#biggest palindrome
def biggestpalindrome(a="QWEREWERET"):
    a=a.lower()
    out=[]
    for start in xrange(len(a)):
        for end in xrange(start+1,len(a)):
            temp=a[start:end]
            if temp==temp[::-1]:
                out.append(temp)
    return out[out.index(max(out,key=len))]
biggestpalindrome()    
    

'erewere'

In [5]:
def quicksort(array=[23,4,2,54,2,54,21,6,8]):
    if len(array)<=1: return array
    left,right,equal=[],[],[]
    equal.append(array[0])
    for i in xrange(1,len(array)):
        if array[i]<equal[0]:
            left.append(array[i])
        elif array[i]>equal[0]:
            right.append(array[i])
        else:
            equal.append(array[i])
    return quicksort(left)+equal+quicksort(right)
print quicksort()

[2, 2, 4, 6, 8, 21, 23, 54, 54]


In [3]:
#mergesort:
#worstcase O(nlogn), average case O(nlogn)
def merge(left,right):
    out = []
    while left and right:
        if left[0] <= right[0]:
            out.append(left[0])
            left=left[1:]
        else:
            out.append(right[0])
            right=right[1:]
    out+=left+right #left or right will be empty list
    return out
            
            
def mergesort(a):
    if len(a)<=1:return a
    return merge(mergesort(a[:len(a)/2]),mergesort(a[len(a)/2:]))
    

print mergesort([23,4,2,54,2,54,21,6,8])

[2, 2, 4, 6, 8, 21, 23, 54, 54]


In [12]:
#insertionsort 
#best O(n), worst O(n^2), average O(n^2)
#steps:
#    starting with first and second value, consecutive comparisons
#    for each i as the highest index cut-off brings the smallest value to the leftmost point
def insertionsort(array=[23,4,2,54,2,54,21,6,8]):
    for i in xrange(1,len(array)):
        for j in xrange(i,0,-1):
            if array[j]<array[j-1]:
                array[j],array[j-1]=array[j-1],array[j]
            else:
                break
    return array
print insertionsort()

[2, 2, 4, 6, 8, 21, 23, 54, 54]


In [20]:
#bubblesort 
#steps:
#    consecutive comparison
#   for each i, bring the highest value to the rightmost point
#best O(n), worst O(n^2), average O(n^2)
def bubblesort(array=[23,4,2,54,2,54,21,6,8]):
    for i in xrange(1,len(array)):
        for j in xrange(1,len(array)-i+1):
            if array[j]<array[j-1]:
                array[j],array[j-1]=array[j-1],array[j]
            else:
                pass
    return array
print bubblesort()
            

[2, 2, 4, 6, 8, 21, 23, 54, 54]


In [146]:
#quicksort with left,right and equal lists
#best O(n), worst O(n^2), average O(nlogn)
def quicksort(array=[23,4,2,54,2,54,21,6,8]):
    if len(array)<=1: return array
    left,equal,right=[],[],[]
    equal.append(array[0])
    for i in xrange(1,len(array)):
        if array[i]<equal[0]:left.append(array[i])
        elif array[i]==equal[0]:equal.append(array[i])
        else: right.append(array[i])
    return quicksort(left)+equal+quicksort(right)    
    
print quicksort()
print quicksort("ASDASD12373")

[2, 2, 4, 6, 8, 21, 23, 54, 54]
['1', '2', '3', '3', '7', 'A', 'A', 'D', 'D', 'S', 'S']


In [95]:
#quicksort in-place
#best O(n), worst O(n^2), average O(nlogn)
def partition(array,lo,hi):
    pivot=array[hi]
    i=lo-1
    for j in xrange(lo,hi):
        if array[j]<pivot:
            i+=1
            array[i],array[j]=array[j],array[i]
    array[i+1],array[hi]=array[hi],array[i+1]
    return i+1
def quicksort(array,lo,hi):
    if lo<hi:
        p=partition(array,lo,hi)
        quicksort(array,lo,p-1)
        quicksort(array,p+1,hi)
array=[23,4,2,54,2,54,21,6,8]
quicksort(array,0,8)
array

[2, 2, 4, 6, 8, 21, 23, 54, 54]

In [114]:
class Node:
    def __init__(self,val):
        self.v=val
        self.l=None
        self.r=None
class BinaryTree:
    def __init__(self):
        self.root=None
    def add(self,val):
        if not self.root:
            self.root = Node(val)
        else:
            self._add(self.root,val)
    def _add(self, node, val):
        if val < node.v:
            if node.l:
                self._add(node.l,val)
            else:
                node.l=Node(val)
        if val >= node.v:
            if node.r:
                self._add(node.r,val)
            else:
                node.r=Node(val)
    def find(self,val):
        if self.root:
            return self._find(self.root,val)
        else:
            return None
    def _find(self,node,val):
        if val==node.v:
            print "equal"
            return "Found node:",val
        elif val<node.v and node.l:
            print "left"
            return self._find(node.l,val)
        elif val>node.v and node.r:
            print "right"
            return self._find(node.r,val)
    def printTree(self):
        if self.root:
            self._printTree(self.root)
    def _printTree(self,node):
        if node:
            self._printTree(node.l)
            print node.v
            self._printTree(node.r)
            
            

b=BinaryTree()
b.add(4)
b.add(1)
b.add(3)
b.add(5)
b.add(6)
b.add(7)
b.add(8)
b.add(2)
b.add(9)

b.printTree()            
print b.find(5)            
#b.root.r.v

right
equal
('Found node:', 5)


In [137]:
#power function in o(logn)
p_powers = {}
def calcpower(p,q):
    global p_powers
    if q==0:return 1
    if q==1: 
        p_powers[q]=p
        return p_powers[q]
    elif q not in p_powers:
        temp = calcpower(p,q/2)
        p_powers[q] = temp*temp
        return p_powers[q]
    else:
        return p_powers[q]
def getpower(p,q):
    out = 1
    for i,pwr in enumerate(reversed(bin(q)[2:])):
        temp = "1"+"0"*i
        if pwr=='1': out*=calcpower(p,int(temp,2))
    return out
print getpower(3,10)
    

59049


In [143]:
#any number to binary subset
def getbinarysubset(n):
    out = []
    for i,pwr in enumerate(reversed(bin(n)[2:])):
        if pwr=='1':out.append(int("1"+"0"*i,2))
    return out
print getbinarysubset(50)        

[2, 16, 32]


In [151]:
#XOR in python:
bool(1) != bool(0) #'1' and '0' don't work

True

In [155]:
10 ^ 13

7

In [157]:
map(lambda x: x*2,range(5))

[0, 2, 4, 6, 8]

In [162]:
import re
re.findall(r'\D+',"ASD5 qwe\n")

['ASD', ' qwe\n']

In [167]:
len(re.findall(r"[-\s(']*foo[-\s).']+","foo bar (foo) bar foo-bar foo_bar foo'bar bar-foo bar, foo."))


6